<a href="https://colab.research.google.com/github/romidj/Lung-cancer-ML-project/blob/main/LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import the necessary libraries

In [1]:
# Standard libraries
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA

# Model selection & tuning
from sklearn.model_selection import train_test_split, GridSearchCV

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

# Evaluation metrics
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

# Load the dataset

In [ ]:
#Load Our Dataset
train = pd.read_csv('/content/dataset.csv')
train.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 339: invalid start byte

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Desribe the Datafrme

In [ ]:
train.columns

In [ ]:
train.dtypes

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
train.shape

# Cleaning the data

#### Handling Missing Values

In [ ]:
### Drop unusefull features
train = train.drop(columns=['id', 'country', 'occupation_type', 'education_level', 'marital_status', 'diet_type', 'bmi', 'diagnosis_date','end_treatment_date' ], errors='ignore')

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
##Fill Missing Values in "age"
#plot
plt.figure(figsize=(10, 6))
plt.hist(train['age'], bins=30, color='blue', alpha=0.7)
plt.title('Histogram of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Scatter plot for 'age'
plt.figure(figsize=(10, 6))
plt.scatter(train.index, train['age'], color='blue', alpha=0.5)
plt.title('Scatter Plot for Age')
plt.xlabel('Index')
plt.ylabel('Age')
plt.show()

In [ ]:
age_median = train['age'].median()
train['age'] = train['age'].fillna(age_median)

In [ ]:
train['gender'] = train['gender'].str.strip().str.lower()
sns.countplot(x='gender', data=train, palette='Blues')
plt.title('Bar Plot for Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

In [ ]:
# Remplacer les erreurs courantes
train['gender'] = train['gender'].replace({
    'malee': 'male',
    'email': 'male',  # ou 'female' selon le contexte
    'feemale': 'female',
    'femal': 'female',
    'femal': 'female'  # Autres erreurs à traiter
})

In [ ]:
sns.countplot(x='gender', data=train, palette='Blues')
plt.title('Bar Plot for Gender')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.show()

In [ ]:
most_frequent_gender = train['gender'].mode()[0]
train['gender'].fillna(most_frequent_gender, inplace=True)
train.isna().sum()

In [ ]:
# Set the figure size (width, height)
plt.figure(figsize=(12, 6))

# Create the count plot for cancer stages
sns.countplot(x='cancer_stage', data=train, palette='Set1')

# Set title and labels
plt.title('Frequency of Cancer Stages', fontsize=16)
plt.xlabel('Cancer Stage', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Show the plot
plt.show()

In [ ]:
print(train['cancer_stage'].unique())

In [ ]:
# Standardize the 'cancer_stage' column values
train['cancer_stage'] = train['cancer_stage'].replace({
    'Stage I': 'Stage 1', 'stage one': 'Stage 1', 'Stage 1': 'Stage 1',
    'Stage II': 'Stage 2', 'stage two': 'Stage 2', 'Stage 2': 'Stage 2',
    'Stage III': 'Stage 3', 'stage three': 'Stage 3', 'Stage 3': 'Stage 3',
    'Stage IV': 'Stage 4', 'stage four': 'Stage 4', 'Stage 4': 'Stage 4'
})

# Check the standardized values to ensure correct replacement
print(train['cancer_stage'].unique())

In [ ]:
# Nettoyage de la colonne 'cancer_stage'
train['cancer_stage'] = train['cancer_stage'].str.strip()  # Supprime les espaces autour des valeurs
train['cancer_stage'].replace('', None, inplace=True)      # Remplace les chaînes vides par None

In [ ]:
# Set the figure size (width, height)
plt.figure(figsize=(12, 6))

# Create the count plot for cancer stages
sns.countplot(x='cancer_stage', data=train, palette='Set1')

# Set title and labels
plt.title('Frequency of Cancer Stages', fontsize=16)
plt.xlabel('Cancer Stage', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Show the plot
plt.show()

In [ ]:
mode_value = train['cancer_stage'].mode()[0]  # Calcule la mode
train['cancer_stage'].fillna(mode_value, inplace=True)  # Remplit les valeurs manquantes


In [ ]:
train['cancer_stage'].unique()

In [ ]:
train.isna().sum()

In [ ]:
train['smoking_status'].unique()

In [ ]:
plt.figure(figsize=(8, 6))
train['smoking_status'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribution of Smoking Status')
plt.xlabel('Smoking Status')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
train['smoking_status'] = train['smoking_status'].fillna(train['smoking_status'].mode()[0])
train.isnull().sum()

In [ ]:
# Plot the distribution of cholesterol_level
plt.figure(figsize=(10,6))
sns.histplot(train['cholesterol_level'], kde=True, bins=30, color='blue')
plt.title('Distribution of Cholesterol Levels')
plt.xlabel('Cholesterol Level')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Fill missing values in 'cholesterol_level' with the median
train['cholesterol_level'].dtypes
train['cholesterol_level'] = train['cholesterol_level'].fillna(train['cholesterol_level'].median())
train.isna().sum()

In [ ]:
train['treatment_type'].unique()

In [ ]:
# Uniformiser la casse pour toutes les valeurs de 'treatment_type'
train['treatment_type'] = train['treatment_type'].str.capitalize()

# Remplacer les valeurs incorrectes par les bonnes
train['treatment_type'] = train['treatment_type'].replace({
    'Surgerie': 'Surgery', 'Sergery': 'Surgery', 'Surgry': 'Surgery',
    'Radiaton': 'Radiation', 'Radation': 'Radiation',
    'Chemotharapy': 'Chemotherapy', 'Chemoterapy': 'Chemotherapy',
    'Combimed': 'Combined', 'Combained': 'Combined'
})


plt.figure(figsize=(8, 6))
train['treatment_type'].value_counts().plot(kind='bar', color='skyblue')
plt.title('Distribution of Treatment Type')
plt.xlabel('Treatment Type')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Remplir les valeurs manquantes dans 'treatment_type' avec la mode
train['treatment_type'] = train['treatment_type'].fillna(train['treatment_type'].mode()[0])

In [ ]:
# Créer un dictionnaire de mappage pour uniformiser les valeurs
mapping = {
    '1': 1, 'Yes': 1, 'yes': 1,  # Variations de "oui" deviennent 1
    '0': 0, 'No': 0, 'no': 0     # Variations de "non" deviennent 0
}

# Appliquer le mappage à la colonne 'cirrhosis'
train['cirrhosis'] = train['cirrhosis'].replace(mapping)

# Vérifier les valeurs uniques après conversion
print(train['cirrhosis'].unique())

In [ ]:
print(train['comorbidity_score'].dtype)

In [ ]:
train['comorbidity_score'].unique()

In [ ]:
train['comorbidity_score'] = pd.to_numeric(train['comorbidity_score'], errors='coerce')
train['comorbidity_score'] = train['comorbidity_score'].fillna(train['comorbidity_score'].median())
print(train.isnull().sum())

In [ ]:
train["other_cancer"].unique()

In [ ]:
# Créer un dictionnaire de mappage pour uniformiser les valeurs
mapping = {
    '0': 0, '1': 1,   # Garde '0' et '1' comme entiers
    'NON': 0, 'no': 0, 'No': 0,  # Toutes les variations de "non" deviennent 0
    'yes': 1, 'Yes': 1           # Toutes les variations de "yes" deviennent 1
}

# Appliquer le mappage à la colonne 'other_cancer'
train['other_cancer'] = train['other_cancer'].replace(mapping)

# Vérifier les valeurs uniques après conversion
print(train['other_cancer'].unique())

In [ ]:
train.head()

In [ ]:
train["family_history"].unique()

In [ ]:
# Créer un dictionnaire de mappage pour uniformiser les valeurs
mapping = {
    '1': 1, 'true': 1, 'Yes': 1, 'yes': 1,  # Variations de "true" ou "yes" deviennent 1
    '0': 0, 'False': 0, 'No': 0, 'no': 0    # Variations de "false" ou "no" deviennent 0
}

# Appliquer le mappage à la colonne 'family_history'
train['family_history'] = train['family_history'].replace(mapping)

# Vérifier les valeurs uniques après conversion
print(train['family_history'].unique())


In [ ]:
train.isnull().sum()

In [ ]:
train["cirrhosis"].unique()

In [ ]:
train = train.dropna()

In [ ]:
train.isna().sum()

In [ ]:
train = train.drop_duplicates()
train.shape
train.head()

# Encoding Categorical Data

In [ ]:
train.columns

In [ ]:
train['treatment_type'].unique()

In [ ]:
#Apply one hot encoding directly on train for the treatment_type column with inplace true
train = pd.get_dummies(train, columns=['treatment_type'], prefix=['treatment_type'], drop_first=True)

In [ ]:
train.head()

In [ ]:
# # Créer un dictionnaire pour mapper les types de traitement
# treatment_mapping = {
#     'Surgery': 1,
#     'Radiation': 2,
#     'radiation': 2,  # Corriger les incohérences (comme la casse)
#     'Combined': 3,
#     'Chemotherapy': 4
# }

# # Appliquer l'encodage
# train['treatment_type_encoded'] = train['treatment_type'].map(treatment_mapping)

# # Vérifier les résultats
# print(train[['treatment_type', 'treatment_type_encoded']].head())

In [ ]:
train['gender'].unique()

In [ ]:
train['gender'] = train['gender'].map({'female': 0, 'male': 1})

In [ ]:
train['cancer_stage'].unique()

In [ ]:
# Map the 'cancer_stage' column to numerical values
train['cancer_stage'] = train['cancer_stage'].map({
    'Stage 1': 1,
    'Stage 2': 2,
    'Stage 3': 3,
    'Stage 4': 4
})

# Display the updated DataFrame
print(train[['cancer_stage']].head())

In [ ]:
train["smoking_status"].unique()


In [ ]:
# Définir un mapping manuel
smoking_mapping = {
    'Never Smoked': 0,
    'Former Smoker': 1,
    'Light Smoker (< 10/day)': 2,
    'Moderate Smoker (10-20/day)': 3,
    'Occasional Smoker': 4,
    'Heavy Smoker (> 20/day)': 5
}

# Remplacer les valeurs dans la colonne
train['smoking_status'] = train['smoking_status'].map(smoking_mapping)

# Afficher le DataFrame
train.head()

In [ ]:
print(type(train))


In [ ]:
# Compter les valeurs uniques pour les colonnes catégoriques
# Convert the NumPy array back into a Pandas DataFrame to use nunique()
train_df = pd.DataFrame(train)
print(train_df.nunique()) # Now you can use nunique()

In [ ]:
# Distribution de la survie par genre
gender_survival = train.groupby('gender')['survived'].value_counts(normalize=True).unstack()
print(gender_survival)

# Visualisation
plt.figure(figsize=(8, 6))
sns.barplot(x='gender', y='survived', data=train, ci=None, palette='Set2')
plt.title('Taux de survie par genre')
plt.xlabel('Genre (0 = Homme, 1 = Femme)')
plt.ylabel('Taux de survie')
plt.show()


Interprétation possible :

Les femmes pourraient avoir des facteurs de risque plus élevés ou recevoir des traitements moins efficaces.
Les stades de cancer chez les femmes pourraient être plus avancés au moment du diagnostic.

In [ ]:
# Distribution de la survie par statut de fumeur
smoking_survival = train.groupby('smoking_status')['survived'].value_counts(normalize=True).unstack()
print(smoking_survival)

# Visualisation
plt.figure(figsize=(10, 6))
sns.barplot(x='smoking_status', y='survived', data=train, ci=None, palette='coolwarm')
plt.title('Taux de survie par statut de fumeur')
plt.xlabel('Statut de fumeur')
plt.ylabel('Taux de survie')
plt.show()


Interprétation possible de ces résultats :
Non-fumeurs (0) ont un taux de survie plus élevé que les fumeurs légers (1), modérés (2), etc.
Fumeurs modérés et lourds (comme 4, 5, 3) pourraient avoir des taux de survie plus faibles. Cela peut être dû à des complications liées au tabagisme, par exemple, qui affectent la santé globale, rendant les patients plus vulnérables à des traitements moins efficaces.

In [ ]:
plt.figure(figsize=(8, 6))
sns.boxplot(x='survived', y='comorbidity_score', data=train, palette='pastel')
plt.title('Score de comorbidité par statut de survie')
plt.xlabel('Survived (0 = Non, 1 = Oui)')
plt.ylabel('Score de comorbidité')
plt.show()


In [ ]:
# # Distribution de la survie par type de traitement
# treatment_survival = train.groupby('treatment_type')['survived'].mean()
# print(treatment_survival)

# # Visualisation
# plt.figure(figsize=(10, 6))
# sns.barplot(x='treatment_type', y='survived', data=train, ci=None, palette='viridis')
# plt.title('Taux de survie par type de traitement')
# plt.xlabel('Type de traitement')
# plt.ylabel('Taux de survie')
# plt.xticks(rotation=45)
# plt.show()


Interprétation possible :
Traitement 3 (Combined) a le taux de survie le plus élevé. Cela pourrait indiquer qu'un traitement combiné (qui inclut plusieurs modalités de traitement) est plus efficace que les traitements individuels dans ce jeu de données.
Traitement 2 (Radiation) et Traitement 1 (Surgery) viennent ensuite, suggérant que la combinaison de radiation ou de chirurgie, même si elles sont efficaces, ne sont pas aussi efficaces que le traitement combiné.
Traitement 4 (Chemotherapy) a le taux de survie le plus bas, ce qui pourrait suggérer que, dans ce contexte particulier, la chimiothérapie n'a pas été aussi efficace, ou bien que les patients ayant reçu ce traitement étaient plus gravement malades au départ.

In [ ]:
# Asthme
asthma_survival = train.groupby('asthma')['survived'].mean()
print("Taux de survie par asthme :\n", asthma_survival)

# Hypertension
hypertension_survival = train.groupby('hypertension')['survived'].mean()
print("Taux de survie par hypertension :\n", hypertension_survival)

# Visualisation combinée
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.barplot(x='asthma', y='survived', data=train, ci=None, palette='Blues')
plt.title('Taux de survie par asthme')
plt.xlabel('Asthme (0 = Non, 1 = Oui)')
plt.ylabel('Taux de survie')

plt.subplot(1, 2, 2)
sns.barplot(x='hypertension', y='survived', data=train, ci=None, palette='Greens')
plt.title('Taux de survie par hypertension')
plt.xlabel('Hypertension (0 = Non, 1 = Oui)')
plt.ylabel('Taux de survie')
plt.tight_layout()
plt.show()


Asthme :
Taux de survie pour ceux sans asthme (asthma=0) est plus élevé que pour ceux ayant de l'asthme (asthma=1).
Cela pourrait suggérer que l'absence d'asthme est associée à un taux de survie plus élevé, ce qui pourrait être dû à des complications liées à l'asthme qui rendent les patients plus vulnérables.
Interprétation :
Cela peut sembler surprenant au premier abord, mais voici quelques explications possibles :
Soins médicaux plus rigoureux : Les patients ayant de l'hypertension peuvent bénéficier d'une surveillance médicale plus étroite, avec des médicaments prescrits et des consultations régulières pour contrôler leur pression artérielle, ce qui pourrait améliorer leur état général et augmenter leur survie.
Autres comorbidités ou traitements associés : Les patients atteints d'hypertension peuvent avoir des traitements ou des soins spécifiques qui contribuent également à leur survie, comme des traitements pour d'autres problèmes cardiovasculaires ou des interventions spécifiques à l'hypertension.
Facteurs d'âge ou de mode de vie : Les personnes plus âgées ou ayant d'autres conditions peuvent aussi être diagnostiquées avec de l'hypertension, et ces facteurs peuvent interagir avec d'autres aspects de leur santé pour influencer leur taux de survie.

# Normalize the data

In [ ]:
### Plot of distribution of data
train.hist(bins=50,figsize=(20,15))
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#Initialize MinMaxScaler
scaler = MinMaxScaler()
#Apply for all the data
train= pd.DataFrame(scaler.fit_transform(train),
                           columns=train.columns,
                           index=train.index)


# Data modeling:

In [ ]:
# Sélectionner les colonnes pertinentes
features = train.drop(columns=['survived'])
target = train['survived']

# S'assurer qu'il n'y a pas de valeurs manquantes
features = features.fillna(0)

# Vérification des dimensions
print(features.shape, target.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

print("Dimensions de l'ensemble d'entraînement :", X_train.shape, y_train.shape)
print("Dimensions de l'ensemble de test :", X_test.shape, y_test.shape)

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Tester plusieurs valeurs de k
errors = []
k_values = list(range(1, 31))

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    pred_k = knn.predict(X_test)
    acc = accuracy_score(y_test, pred_k)
    errors.append(1 - acc)  # erreur = 1 - accuracy

# Graphique
plt.figure(figsize=(10,6))
plt.plot(k_values, errors, color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=8)
plt.title('Erreur vs. Valeur de K')
plt.xlabel('K')
plt.ylabel('Erreur')
plt.grid(True)
plt.show()

In [ ]:
best_k = k_values[np.argmin(errors)]
print("Best k:", best_k)

knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import roc_curve, auc

# Probabilités pour ROC
y_prob = knn.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Plot ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - KNN')
plt.legend(loc="lower right")
plt.grid()
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# Create GridSearchCV object
grid_search = GridSearchCV(KNeighborsClassifier(n_neighbors=best_k), param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Get the best model
best_knn_model = grid_search.best_estimator_

In [ ]:
#print the results using the best hyperparameters
y_pred_best = best_knn_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)

print(f"Accuracy: {accuracy_best}")
print(f"Precision: {precision_best}")
print(f"Recall: {recall_best}")
print(f"F1-Score: {f1_best}")

In [ ]:
#conf matrix
conf_matrix_best = confusion_matrix(y_test, y_pred_best)
print(f"Confusion Matrix:\n{conf_matrix_best}")

## Random Forest

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.model_selection import learning_curve, GridSearchCV  # Pour validation croisee et recherche d'hyperparamètres
# Initialiser le modèle
model = RandomForestClassifier(random_state=42)

# Entraîner le modèle
model.fit(X_train, y_train)



In [ ]:
# ==================== Cellule 1 : Recherche des meilleurs hyperparamètres ====================

# Définir la grille des hyperparamètres à tester
param_grid = {
    'n_estimators': [100, 200, 300],  # Nombre d'arbres dans la forêt
    'max_depth': [None, 10, 20, 30],  # Profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],  # Minimum d'échantillons pour diviser un noeud
    'min_samples_leaf': [1, 2, 4]  # Minimum d'échantillons dans une feuille
}

# Initialiser un modèle de base RandomForest
base_model = RandomForestClassifier(random_state=42)

# Utiliser GridSearchCV pour tester toutes les combinaisons possibles
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=1, scoring='accuracy')

# Lancer la recherche sur les données d'entraînement
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres trouvés
print("Meilleurs paramètres trouvés :", grid_search.best_params_)



In [ ]:
# ==================== Cellule 2 : Entraînement du meilleur modèle ====================

# Récupérer le meilleur modèle obtenu
best_model = grid_search.best_estimator_

# Entraîner le modèle optimisé sur tout l'ensemble d'entraînement
best_model.fit(X_train, y_train)

In [ ]:
# ==================== Cellule 3 : Prédiction et évaluation ====================

# Prédire les étiquettes sur l'ensemble de test
y_pred = best_model.predict(X_test)

# Calculer l'exactitude (Accuracy)
accuracy = accuracy_score(y_test, y_pred)

# Créer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# Générer un rapport de classification
class_report = classification_report(y_test, y_pred)

# Afficher les résultats
print("\nExactitude (Accuracy) :", accuracy)
print("\nMatrice de confusion :\n", conf_matrix)
print("\nRapport de classification :\n", class_report)

In [ ]:
# Calculer la courbe d'apprentissage (train/validation)
train_sizes, train_scores, valid_scores = learning_curve(
    best_model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10), random_state=42
)

# Calculer la moyenne et l'écart-type pour chaque taille d'échantillon
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
valid_scores_mean = np.mean(valid_scores, axis=1)
valid_scores_std = np.std(valid_scores, axis=1)

# Créer le graphique
plt.figure(figsize=(10,6))
plt.title("Courbe d'apprentissage du Random Forest optimisé")
plt.xlabel("Taille de l'ensemble d'apprentissage")
plt.ylabel("Exactitude (Accuracy)")
plt.grid()

# Remplir entre les intervalles (moyenne ± std) pour visualiser l'incertitude
plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.1, color="r")
plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std,
                 valid_scores_mean + valid_scores_std, alpha=0.1, color="g")

# Tracer la courbe de performance sur les données d'entraînement
plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Score d'entraînement")

# Tracer la courbe de performance sur les données de validation croisée
plt.plot(train_sizes, valid_scores_mean, 'o-', color="g", label="Score de validation croisée")

# Ajouter la légende
plt.legend(loc="best")

# Afficher la courbe
plt.show()

#SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, f1_score

kernels = ['linear', 'rbf', 'poly', 'sigmoid']
f1_scores = {}

for kernel in kernels:
    print(f"\n--- Kernel: {kernel} ---")

    # Si kernel='poly', on essaiera plusieurs degrés après
    # Only set degree for 'poly' kernel, otherwise leave it to default
    if kernel == 'poly':
        degree_values = [2, 3, 4]
        for degree in degree_values:
            print(f"\n  > Degree: {degree}")
            clf = SVC(kernel=kernel, degree=degree)
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)

            print(classification_report(y_test, y_pred))
            f1_scores[f"{kernel}_deg{degree}"] = f1_score(y_test, y_pred, average='binary')  # or 'weighted'/'binary' as appropriate
    else:
        clf = SVC(kernel=kernel)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    f1_scores[kernel] = f1_score(y_test, y_pred)


--- Kernel: linear ---
              precision    recall  f1-score   support

         0.0       0.86      0.88      0.87     52405
         1.0       0.80      0.79      0.79     33995

    accuracy                           0.84     86400
   macro avg       0.83      0.83      0.83     86400
weighted avg       0.84      0.84      0.84     86400


--- Kernel: rbf ---


In [ ]:
f1_summary = pd.DataFrame.from_dict(f1_scores, orient='index', columns=['F1 Score'])
f1_summary = f1_summary.sort_values(by='F1 Score', ascending=False)
print("\nRésumé des F1-scores :\n", f1_summary)

In [ ]:
# Simple F1 score comparison plot
plt.figure(figsize=(10, 5))
plt.bar(f1_scores.keys(), f1_scores.values(), color='skyblue')
plt.title('F1 Score Comparison of SVM Kernels')
plt.ylabel('F1 Score')
plt.xlabel('Kernel / Degree')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Apply PCA to reduce to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_train)
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_train, test_size=0.3, random_state=42)

# Fit the SVM with a linear kernel
clf = SVC(kernel='linear', C=1)
clf.fit(X_train, y_train)

In [ ]:
# Plot decision boundary
def plot_svm_boundary(X, y, model, title):
    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, cmap='coolwarm', alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', edgecolors='k')
    plt.title(title)
    plt.xlabel('PCA 1')
    plt.ylabel('PCA 2')
    plt.grid(True)
    plt.show()

plot_svm_boundary(X_test, y_test, clf, title="SVM Linear Kernel Decision Boundary")

In [ ]:
# Train the best model again (poly with degree 4)
best_clf = SVC(kernel='poly', degree=4)
best_clf.fit(X_train, y_train)
y_pred = best_clf.predict(X_test)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display it
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
plt.title('Confusion Matrix - SVM (poly, degree=4)')
plt.show()

# XGBOOST


In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# Fit
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curve

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_prob))

# **Logistic Regression**

In [ ]:
# Step 1: Save your DataFrame to a CSV file
train.to_csv('modified_dataset.csv', index=False)

# Step 2: Download it to your local machine
from google.colab import files
files.download('modified_dataset.csv')


We run a heatmap to see if logistic regression is going to performgood results on our dataset, through the correlation between the dumy vaiables and the target variable


In [ ]:
train.head()

In [ ]:
corr = train.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

we notice that the target variable "survived" is highly correlated with the features : cancer stage, smoking status and family history (negatively), and highly positively correlated to the treatement type combined
The features: treatement_type_combined is highly correlated with treatement_type_surgery and treatement_type_Radiation what may cnfuse the  logreg model, so since we can  conclude treatement_type_combined  through the other features we'll combine all of them into one catigorical column having tha values ( surgery , radiation and combined )

In [ ]:
train.head()

In [ ]:
features1 = features.copy()

In [ ]:
features1.drop(columns=['treatment_type_Combined'], inplace=True)

In [ ]:
corr = features1.corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
X1_train, X1_test, y_train, y_test = train_test_split(features1, target, test_size=0.2, random_state=42)

print("Dimensions de l'ensemble d'entraînement :", X1_train.shape, y_train.shape)
print("Dimensions de l'ensemble de test :", X1_test.shape, y_test.shape)

In [ ]:
logreg = LogisticRegression(max_iter=1000)

logreg.fit(X1_train, y_train)

# Prédire sur l'ensemble de test
y_pred = logreg.predict(X1_test)
y_prob = logreg.predict_proba(X1_test)[:, 1]

In [ ]:
y_pred

In [ ]:
# Assuming you have a trained model called 'model'
print("Coefficients: ", logreg.coef_)
print("Intercept: ", logreg.intercept_)


In [ ]:
print("Classification Report :\n", classification_report(y_test, y_pred))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Prédiction")
plt.ylabel("Vérité terrain")
plt.title("Matrice de confusion")
plt.show()

In [ ]:
# Courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
auc_score = roc_auc_score(y_test, y_prob)

plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("Taux de faux positifs")
plt.ylabel("Taux de vrais positifs")
plt.title("Courbe ROC")
plt.legend()
plt.show()
